# Fetching Dataset from Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
shutil.copy("/content/drive/MyDrive/dataset.npz", "Dataset")

Mounted at /content/drive


'Dataset'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importing Libraries

In [4]:
import os
import time
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.random import randint
from tensorflow.keras import Input
from numpy import load, zeros, ones
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Conv2D, UpSampling2D, LeakyReLU, Activation
from tensorflow.keras.layers import Concatenate, Dropout, BatchNormalization, LeakyReLU

# Creating Functions

## Defining Discriminator

In [5]:
def define_discriminator(image_shape):
	init = RandomNormal(stddev=0.02)

	in_src_image = Input(shape=image_shape)
	in_target_image = Input(shape=image_shape)

	merged = Concatenate()([in_src_image, in_target_image])

	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)

	model = Model([in_src_image, in_target_image], patch_out)

	optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, loss_weights=[0.5])
	return model

## Defining Generator

In [6]:
def define_generator(image_shape = (256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)

    g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
    g = BatchNormalization()(g, training=True)
    g3 = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g3)
    g = BatchNormalization()(g, training=True)
    g2 = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g2)
    g = BatchNormalization()(g, training=True)
    g1 = LeakyReLU(alpha=0.2)(g)

    for _ in range(6):
        g = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(g1)
        g = BatchNormalization()(g, training=True)
        g = LeakyReLU(alpha=0.2)(g)

        g = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(g)
        g = BatchNormalization()(g, training=True)

        g1 = Concatenate()([g, g1])

    g = UpSampling2D((2, 2))(g1)
    g = Conv2D(128, (1, 1), kernel_initializer=init)(g)
    g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, g2])
    g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)

    g = UpSampling2D((2, 2))(g)
    g = Conv2D(64, (1, 1), kernel_initializer=init)(g)
    g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, g3])
    g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g = BatchNormalization()(g, training=True)
    out_image = Activation('tanh')(g)

    model = Model(in_image, out_image)
    return model

## Initializing GAN training

In [7]:
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=optimizer, loss_weights=[1,100])
	return model

## Loading Real Samples

In [8]:

def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X2, X1]

## Generating Real Fake Samples

In [9]:
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

## Generating Fake Samples

In [10]:
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

## Summarizing Training and Saving Model

In [11]:
def summarize_performance(step, g_model, d_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    plt.figure(figsize=(14, 14))
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.title('Low-Light')
        plt.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.title('Generated')
        plt.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.title('Ground Truth')
        plt.imshow(X_realB[i])
    # save plot to file
    filename1 = step_output + 'plot_%06d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
    # save the generator model
    filename2 = model_output + 'gen_model_%06d.h5' % (step+1)
    g_model.save(filename2)
    # save the discriminator model
    filename3 = model_output + 'disc_model_%06d.h5' % (step+1)
    d_model.save(filename3)
    print('[.] Saved Step : %s' % (filename1))
    print('[.] Saved Model: %s' % (filename2))
    print('[.] Saved Model: %s' % (filename3))

## Training Function

In [11]:
def train(d_model, g_model, gan_model, dataset, n_epochs=50, batch_size=12):
    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / batch_size)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print("[!] Number of steps {}".format(n_steps))
    print("[!] Saves model/step output at every {}".format(bat_per_epo * 1))
    # manually enumerate epochs
    for i in range(n_steps):
        start = time.time()
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, batch_size, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        # summarize performance
        time_taken = time.time() - start
        print(
            '[*] %06d, d1[%.3f] d2[%.3f] g[%06.3f] ---> time[%.2f], time_left[%.08s]'
                %
            (i+1, d_loss1, d_loss2, g_loss, time_taken, str(datetime.timedelta(seconds=((time_taken) * (n_steps - (i + 1))))).split('.')[0].zfill(8))
        )
        # summarize model performance
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, d_model, dataset)

# Main Function

## Loading Dataset

In [12]:
dataset = load_real_samples('Dataset')
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]

Loaded (485, 256, 256, 3) (485, 256, 256, 3)


## Defining Models

In [13]:
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
gan_model = define_gan(g_model, d_model, image_shape)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


## Creating model Directory and Calling Train Function

In [15]:
dir = '/models/'
fileName = 'Enhancement Model'
step_output = dir + fileName + "/Step Output/"
model_output = dir + fileName + "/Model Output/"
if not os.path.exists(dir):
    os.makedirs(dir)
if fileName not in os.listdir(dir):
    os.mkdir(dir + fileName)
    os.mkdir(step_output)
    os.mkdir(model_output)

train(d_model, g_model, gan_model, dataset, batch_size=12)

[!] Number of steps 2000
[!] Saves model/step output at every 40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
[*] 000001, d1[0.428] d2[0.428] g[22.504] ---> time[1.77], time_left[00:58:49]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
[*] 000002, d1[0.428] d2[0.428] g[22.503] ---> time[1.49], time_left[00:49:35]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
[*] 000003, d1[0.428] d2[0.428] g[22.502] ---> time[1.47], time_left[00:49:05]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
[*] 000004, d1[0.428] d2[0.428] g[22.503] ---> time[1.50], time_left[00:49:50]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
[*] 000005, d1[0.428] d2[0.429] g[22.502] ---> time[1.49], time_left[00:49:32]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
[*] 000006, d1[0.428] d2[0.429] g[22.499] ---> time[1.53], time_left[00:50:46]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
[*] 000007, d1[0.428] d2[0.429] g[22.497] ---> time[1.80], time_left[00:59:42]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
[*] 000008, d1[0.428] d2[0.429] g[22.496] ---> time[1.76], time_left[0

[.] Saved Step : /models/Enhancement Model/Step Output/plot_000040.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_000040.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_000040.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
[*] 000041, d1[0.429] d2[0.429] g[22.459] ---> time[1.82], time_left[00:59:24]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
[*] 000042, d1[0.429] d2[0.429] g[22.458] ---> time[1.52], time_left[00:49:38]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
[*] 000043, d1[0.429] d2[0.429] g[22.456] ---> time[1.80], time_left[00:58:45]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
[*] 000044, d1[0.429] d2[0.429] g[22.456] ---> time[1.55], time_left[00:50:35]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
[*] 000045, d1[0.429] d2[0.429] g[22.453] ---> time[1.52], time_left[00:49:36]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
[*] 000046, d1[0.429] d2[0.429] g[22.453] ---> time[1.80], time_left[00:58:37]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
[*] 000047, d1[0.429] d2[0.42

[.] Saved Step : /models/Enhancement Model/Step Output/plot_000080.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_000080.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_000080.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
[*] 000081, d1[0.429] d2[0.429] g[22.425] ---> time[1.59], time_left[00:50:52]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
[*] 000082, d1[0.429] d2[0.429] g[22.425] ---> time[1.58], time_left[00:50:23]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
[*] 000083, d1[0.429] d2[0.429] g[22.423] ---> time[1.58], time_left[00:50:22]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
[*] 000084, d1[0.429] d2[0.429] g[22.422] ---> time[1.57], time_left[00:50:06]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
[*] 000085, d1[0.429] d2[0.429] g[22.424] ---> time[1.57], time_left[00:50:01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
[*] 000086, d1[0.429] d2[0.429] g[22.422] ---> time[1.83], time_left[00:58:28]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
[*] 000087, d1[0.429] d2[0.42

[.] Saved Step : /models/Enhancement Model/Step Output/plot_000120.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_000120.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_000120.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
[*] 000121, d1[0.429] d2[0.429] g[22.395] ---> time[1.84], time_left[00:57:38]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
[*] 000122, d1[0.429] d2[0.429] g[22.395] ---> time[1.85], time_left[00:57:51]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step
[*] 000123, d1[0.429] d2[0.429] g[22.392] ---> time[1.85], time_left[00:58:01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
[*] 000124, d1[0.429] d2[0.429] g[22.391] ---> time[1.87], time_left[00:58:20]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
[*] 000125, d1[0.429] d2[0.429] g[22.391] ---> time[1.59], time_left[00:49:34]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
[*] 000126, d1[0.429] d2[0.429] g[22.390] ---> time[1.58], time_left[00:49:25]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
[*] 000127, d1[0.429] d2[0.42

[.] Saved Step : /models/Enhancement Model/Step Output/plot_000160.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_000160.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_000160.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
[*] 000161, d1[0.429] d2[0.429] g[22.354] ---> time[1.62], time_left[00:49:44]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
[*] 000162, d1[0.429] d2[0.429] g[22.353] ---> time[1.85], time_left[00:56:31]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
[*] 000163, d1[0.429] d2[0.429] g[22.352] ---> time[1.57], time_left[00:48:10]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
[*] 000164, d1[0.429] d2[0.429] g[22.350] ---> time[1.86], time_left[00:56:50]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
[*] 000165, d1[0.429] d2[0.429] g[22.349] ---> time[1.57], time_left[00:48:09]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
[*] 000166, d1[0.429] d2[0.429] g[22.350] ---> time[1.58], time_left[00:48:20]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
[*] 000167, d1[0.429] d2[0.42

...

[.] Saved Step : /models/Enhancement Model/Step Output/plot_001840.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_001840.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_001840.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step
[*] 001841, d1[0.435] d2[0.435] g[21.279] ---> time[1.84], time_left[00:04:53]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
[*] 001842, d1[0.435] d2[0.435] g[21.279] ---> time[1.60], time_left[00:04:12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
[*] 001843, d1[0.435] d2[0.435] g[21.279] ---> time[1.85], time_left[00:04:49]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
[*] 001844, d1[0.435] d2[0.435] g[21.278] ---> time[1.84], time_left[00:04:47]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
[*] 001845, d1[0.435] d2[0.435] g[21.278] ---> time[1.58], time_left[00:04:05]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
[*] 001846, d1[0.435] d2[0.435] g[21.278] ---> time[1.86], time_left[00:04:45]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
[*] 001847, d1[0.435] d2[0.43

[.] Saved Step : /models/Enhancement Model/Step Output/plot_001880.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_001880.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_001880.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
[*] 001881, d1[0.435] d2[0.435] g[21.260] ---> time[1.59], time_left[00:03:09]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step
[*] 001882, d1[0.435] d2[0.435] g[21.261] ---> time[1.59], time_left[00:03:08]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
[*] 001883, d1[0.435] d2[0.435] g[21.261] ---> time[1.63], time_left[00:03:10]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step
[*] 001884, d1[0.435] d2[0.435] g[21.260] ---> time[1.86], time_left[00:03:36]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
[*] 001885, d1[0.435] d2[0.435] g[21.260] ---> time[1.58], time_left[00:03:01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
[*] 001886, d1[0.435] d2[0.435] g[21.260] ---> time[1.83], time_left[00:03:29]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
[*] 001887, d1[0.435] d2[0.43

[.] Saved Step : /models/Enhancement Model/Step Output/plot_001920.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_001920.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_001920.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
[*] 001921, d1[0.435] d2[0.435] g[21.236] ---> time[1.87], time_left[00:02:27]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
[*] 001922, d1[0.435] d2[0.435] g[21.237] ---> time[1.84], time_left[00:02:23]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
[*] 001923, d1[0.435] d2[0.435] g[21.236] ---> time[1.57], time_left[00:02:01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step
[*] 001924, d1[0.435] d2[0.435] g[21.236] ---> time[1.59], time_left[00:02:00]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
[*] 001925, d1[0.435] d2[0.435] g[21.235] ---> time[1.58], time_left[00:01:58]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
[*] 001926, d1[0.435] d2[0.435] g[21.233] ---> time[1.85], time_left[00:02:16]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
[*] 001927, d1[0.435] d2[0.43

[.] Saved Step : /models/Enhancement Model/Step Output/plot_001960.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_001960.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_001960.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
[*] 001961, d1[0.435] d2[0.435] g[21.215] ---> time[1.57], time_left[00:01:01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
[*] 001962, d1[0.435] d2[0.435] g[21.216] ---> time[1.84], time_left[00:01:10]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
[*] 001963, d1[0.435] d2[0.435] g[21.216] ---> time[1.85], time_left[00:01:08]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
[*] 001964, d1[0.435] d2[0.435] g[21.215] ---> time[1.64], time_left[00:00:58]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
[*] 001965, d1[0.435] d2[0.435] g[21.214] ---> time[1.60], time_left[00:00:56]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
[*] 001966, d1[0.435] d2[0.435] g[21.215] ---> time[1.84], time_left[00:01:02]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
[*] 001967, d1[0.435] d2[0.43

[.] Saved Step : /models/Enhancement Model/Step Output/plot_002000.png
[.] Saved Model: /models/Enhancement Model/Model Output/gen_model_002000.h5
[.] Saved Model: /models/Enhancement Model/Model Output/disc_model_002000.h5


In [35]:
!ls -lh /models/Enhancement\ Model


total 8.0K
drwxr-xr-x 2 root root 4.0K Mar 28 12:55 'Model Output'
drwxr-xr-x 2 root root 4.0K Mar 28 12:55 'Step Output'


In [36]:
# Define the path in Google Drive
drive_path = "/content/drive/MyDrive/Enhancement_Model"

# Create directory if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Save models
d_model.save(f"{drive_path}/discriminator.h5")
g_model.save(f"{drive_path}/generator.h5")
gan_model.save(f"{drive_path}/gan.h5")

print("Models saved in Google Drive at:", drive_path)


Models saved in Google Drive at: /content/drive/MyDrive/Enhancement_Model


In [37]:
!ls -lh /content/drive/MyDrive/Enhancement_Model

total 307M
-rw------- 1 root root  27M Mar 28 14:50 discriminator.h5
-rw------- 1 root root 217M Mar 28 14:50 gan.h5
-rw------- 1 root root  64M Mar 28 14:50 generator.h5


In [38]:
from google.colab import files

# Define paths to the saved models
drive_path = "/content/drive/MyDrive/Enhancement_Model"

# Download each model
files.download(f"{drive_path}/discriminator.h5")
files.download(f"{drive_path}/generator.h5")
files.download(f"{drive_path}/gan.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>